# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235082839245                   -0.50    8.0         
  2   -7.250344279636       -1.82       -1.40    1.0   12.3ms
  3   -7.251171121974       -3.08       -2.20    1.0   11.7ms
  4   -7.251304490532       -3.87       -2.21    3.0   14.8ms
  5   -7.251336798261       -4.49       -2.71    1.0   11.4ms
  6   -7.251334497700   +   -5.64       -2.84    2.0   13.7ms
  7   -7.251338450767       -5.40       -3.30    1.0   11.1ms
  8   -7.251338787686       -6.47       -4.04    1.0   12.3ms
  9   -7.251338798162       -7.98       -4.48    3.0   15.8ms
 10   -7.251338798555       -9.41       -4.91    1.0   11.9ms
 11   -7.251338798682       -9.89       -5.44    1.0   12.6ms
 12   -7.251338798701      -10.73       -5.71    2.0   14.7ms
 13   -7.251338798704      -11.52       -6.25    1.0   11.8ms
 14   -7.251338798704      -12.73       -6.30    3.0   15.3ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.09334335264061824
[ Info: Arnoldi iteration step 2: normres = 0.586382652906392
[ Info: Arnoldi iteration step 3: normres = 0.6601643835992378
[ Info: Arnoldi iteration step 4: normres = 0.20524597011713683
[ Info: Arnoldi iteration step 5: normres = 0.46366753525519744
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.20e-02, 4.29e-02, 3.77e-01, 2.66e-01, 1.32e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3932623466593355
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.19e-03, 2.78e-01, 2.20e-01, 1.15e-01, 1.16e-01)
[ Info: Arnoldi iteration step 7: normres = 0.08824451837761148
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.87e-04, 2.43e-02, 7.94e-03, 4.41e-02, 6.37e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09239874787271968
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (7.67e-